In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import welch, windows, convolve
from scipy.fft import irfft, rfftfreq

# Load and normalize the noisy speech signal
fs, data = wavfile.read("speech_w_noise.wav")
xn = data / max(abs(data))  
h_len = 128

# Split signal into speech + noise (full signal) and noise-only segments
noise_start = int(0.0 * fs)  # Adjust based on plot from task a
noise_end = int(0.5 * fs)
noise_segment = xn[noise_start:noise_end]

speech_start = int(3.0 * fs)
speech_end = int(4.5 * fs)
speech_segment = xn[speech_start:speech_end]

# Estimate PSD of speech+noise and noise segments using Welch's method
f_filt, P_xx = welch(speech_segment, fs=fs, nperseg=h_len, noverlap=h_len//2)  # PSD of speech + noise
_, P_vv = welch(noise_segment, fs=fs, nperseg=h_len, noverlap=h_len//2)  # PSD of noise-only
f_filt, P_xx = sig.welch(xn, fs=fs, nperseg=h_len, noverlap=h_len//2)  # Calculate PSD of x[n]

# Estimate noise floor based on the median of the total PSD
noise_floor = np.median(P_vv)  # Estimate noise level

# Refine P_vv by applying the noise floor threshold
P_vv = np.where(P_xx < noise_floor * 1.5, P_xx, noise_floor)  # Separate noise PSD

# Calculate the Wiener filter response H_v(omega) = P_ss / (P_ss + P_vv)
P_ss = P_xx - P_vv  # Estimate signal PSD by subtracting noise
H_v = P_ss / (P_ss + P_vv)

# Apply gain cap to limit the maximum value of H_v
H_v_capped = np.minimum(H_v, 1)
H_s = 1 - H_v_capped
D = (h_len - 1) / 2  
frequencies = rfftfreq(h_len, 1/fs)  
H_s_phase_adjusted = H_s * np.exp(-1j * 2 * np.pi * frequencies * D)

h_s = irfft(H_s_phase_adjusted, n=h_len)
h_s_windowed = h_s * windows.hamming(h_len)

# Plotting
plt.figure(figsize=(12, 6))

# High-resolution frequency response plot of |H_s(ω)|
plt.subplot(2, 1, 1)
plt.plot(f_filt, 20 * np.log10(np.abs(H_s)), label=r"$|H_s(\omega)|$")
plt.xlim([0, fs / 2])
plt.title("High-Resolution Magnitude Response $|H_s(\\omega)|$")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude (dB)")
plt.grid(True)
plt.legend()

# Plot the impulse response h_s[n]
plt.subplot(2, 1, 2)
plt.stem(h_s_windowed, linefmt='C1-', markerfmt='C1o', basefmt='k')
plt.title("Impulse Response $h_s[n]$")
plt.xlabel("Sample")
plt.ylabel("Amplitude")
plt.grid(True)

plt.tight_layout()
plt.show()



filtered_signal = convolve(xn, h_s, mode='same')

Audio(filtered_signal, rate=fs)
Audio(noise_segment, rate=fs)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import welch, windows, stft, istft
from IPython.display import Audio
from scipy.fft import irfft, rfftfreq

# Load and normalize the noisy speech signal
fs, data = wavfile.read("speech_w_noise.wav")
xn = data / np.max(np.abs(data))

frame_length = 512
hop_length = frame_length // 4
fft_length = frame_length
oversubtraction_factor = 2
spectral_floor = 0.02

noise_start, noise_end = int(0.0 * fs), int(1.0 * fs)
noise_segment = xn[noise_start:noise_end]
f, t, noise_stft = stft(noise_segment, fs=fs, nperseg=frame_length, noverlap=frame_length-hop_length, nfft=fft_length)
noise_psd = np.mean(np.abs(noise_stft)**2, axis=1)
f, t, signal_stft = stft(xn, fs=fs, nperseg=frame_length, noverlap=frame_length-hop_length, nfft=fft_length)

signal_psd = np.abs(signal_stft)**2
subtracted_psd = np.maximum(signal_psd - oversubtraction_factor * noise_psd[:, np.newaxis], spectral_floor * signal_psd)
enhanced_magnitude = np.sqrt(subtracted_psd)

enhanced_stft = enhanced_magnitude * np.exp(1j * np.angle(signal_stft))
_, enhanced_signal = istft(enhanced_stft, fs=fs, nperseg=frame_length, noverlap=frame_length-hop_length, nfft=fft_length)

# Normalize the enhanced signal
filtered_signal = enhanced_signal / np.max(np.abs(enhanced_signal))

# Calculate SNR improvement
def calculate_snr(signal, noise):
    return 10 * np.log10(np.sum(signal**2) / np.sum(noise**2))

original_snr = calculate_snr(xn[noise_end:], xn[noise_start:noise_end])
enhanced_snr = calculate_snr(filtered_signal[noise_end:], filtered_signal[noise_start:noise_end])
snr_improvement = enhanced_snr - original_snr

# Plotting
plt.figure(figsize=(12, 8))

plt.subplot(4, 1, 1)
plt.plot(np.linspace(0, len(xn)/fs, len(xn), endpoint=False), xn)
plt.title("Original Signal")
plt.xlabel("Time (s)")



plt.subplot(4, 1, 2)
plt.plot(np.linspace(0, len(filtered_signal)/fs, len(filtered_signal), endpoint=False), filtered_signal)
plt.title("Improved Signal")
plt.xlabel("Time (s)")


plt.subplot(4, 1, 3)
plt.plot(f_filt, 20 * np.log10(np.abs(H_s)), label=r"$|H_s(\omega)|$")
plt.xlim([0, fs / 2])
plt.title("High-Resolution Magnitude Response")
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.grid(True)


plt.subplot(4, 1, 4)
plt.stem(h_s_windowed, linefmt='C1-', markerfmt='C1o', basefmt='k')
plt.title("Impulse Response")
plt.xlabel("Sample")
plt.ylabel("Amplitude")
plt.grid(True)


plt.legend()

plt.tight_layout()
plt.show()

print(f"SNR Improvement: {snr_improvement:.2f} dB")

Audio(enhanced_signal, rate=fs)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch, csd
from scipy.fft import irfft, rfftfreq

# Load and normalize the noisy speech signal
fs, data = wavfile.read("speech_w_noise.wav")
xn = data / max(abs(data))  
noise_start = int(0.5 * fs) 
noise_end = int(1.5 * fs)
noise_segment = xn[noise_start:noise_end]

speech_start = int(3.0 * fs)
speech_end = int(4.5 * fs)
speech_segment = xn[speech_start:speech_end]

h_len = 256

#f_xx, S_xx = welch(speech_segment, fs=fs, nperseg=M, nfft=M)  
#f_vv, S_vv = welch(noise_segment, fs=fs, nperseg=M, nfft=M)   

f_filt, P_xx = welch(speech_segment, fs=fs, nperseg=h_len, noverlap=h_len//2)  # Calculate PSD of x[n]
f_filt, P_vv = welch(noise_segment, fs=fs, nperseg=h_len, noverlap=h_len//2)  # Calculate PSD of x[n]



noise_floor = np.median(P_xx)  # Estimate noise level

P_vv = np.where(P_xx < noise_floor*1.5, P_xx, noise_floor)  

#Wiener filter frequency response H_v(omega) = S_xx / S_vv
H_v = P_xx / P_vv

# Apply gain cap to enhance the filter's magnitude response with upper limit of 1
H_v_capped = np.minimum(H_v, 1)

# Calculate the complementary filter frequency response H_s(omega) = 1 - H_v(omega)
H_s = 1 - H_v_capped
D = (h_len - 1) / 2  
frequencies = rfftfreq(h_len, 1/fs)  
H_s_phase_adjusted = H_s * np.exp(-1j * 2 * np.pi * frequencies * D)


h_s = irfft(H_s_phase_adjusted, n=h_len)

# High-resolution frequency response plot for verification
plt.figure(figsize=(12, 6))

# Plot high-resolution magnitude response of H_s
high_res_freqs = rfftfreq(1024, 1/fs)  # High-resolution frequency axis
plt.subplot(2, 1, 1)
plt.plot(frequencies, np.abs(H_s), label=r"$|H_s(\omega)|$")
plt.xlim([0, 10000])
plt.title("High-Resolution Magnitude Response $|H_s(\\omega)|$")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.grid(True)
plt.legend()

# Plot the impulse response h_s[n]
plt.subplot(2, 1, 2)
plt.stem(h_s, linefmt='C1-', markerfmt='C1o', basefmt='k')
plt.title("Impulse Response $h_s[n]$")
plt.xlabel("Sample")
plt.ylabel("Amplitude")
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
#Task 2e:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import firwin, freqz

# Filter specifications
omega_c = np.pi / 4  # -6 dB cutoff frequency in radians
transition_width = np.pi / 10  # Maximum allowable transition width
stopband_threshold = -50  # Required minimum stopband attenuation in dB
window_type = 'boxcar'  # Type of window to use

def design_and_analyze_filter(M):
    """
    Designs an FIR filter with M taps and analyzes its frequency response.
    
    Parameters:
        M (int): Number of filter taps.
        
    Returns:
        freqs (np.ndarray): Array of normalized frequencies.
        magnitude_response (np.ndarray): Magnitude response in dB.
        actual_tw (float): Calculated transition width.
    """
    # Design the FIR filter with the specified cutoff and window type
    h_D = firwin(M, omega_c / np.pi, window=window_type)
    
    # Compute the frequency response
    freqs, H_D = freqz(h_D, worN=128)
    magnitude_response = 20 * np.log10(np.abs(H_D))  # Convert magnitude to dB
    
    # Locate -6 dB (cutoff) and -50 dB (stopband start) points
    passband_edge = np.argmax(magnitude_response <= -6)
    stopband_edge = np.argmax(magnitude_response <= stopband_threshold)
    
    # Calculate the actual transition width
    actual_tw = freqs[stopband_edge] - freqs[passband_edge]
    
    return freqs, magnitude_response, actual_tw

# Find minimum number of taps
M = 5  # Starting point for filter taps
while True:
    freqs, magnitude_response, actual_tw = design_and_analyze_filter(M)
    if actual_tw <= transition_width:
        M_min = M
        break
    M += 1

# Plot frequency response for the final filter with M_min taps
plt.figure(figsize=(10, 6))
plt.plot(freqs / np.pi, magnitude_response, label="Magnitude Response")
plt.axhline(-6, color='red', linestyle='--', label='-6 dB Passband Edge')
plt.axhline(-50, color='green', linestyle='--', label='-50 dB Stopband Edge')
plt.axvline(omega_c / np.pi, color='orange', linestyle='--', label='Cutoff Frequency ($\omega_c$)')
plt.title(f"Frequency Response of Hamming Window Filter (M = {M_min})")
plt.xlabel("Normalized Frequency (×π rad/sample)")
plt.ylabel("Magnitude (dB)")
plt.legend()
plt.grid(True)
plt.ylim([-100, 10])
plt.xlim([0, 1])
plt.show()

print(f"The minimum number of filter taps M_min to meet the transition width requirement is: {M_min}")


In [ ]:
#task 2d: 


"# WRITE YOUR CODE IN THIS CELL:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import firwin, freqz


omega_c = np.pi / 4  
filter_length = 51  

# Define windows to be used in FIR filter design
window_types = {
    'Rectangular': 'boxcar',  # Equivalent to no windowing, leading to high spectral leakage
    'Hamming': 'hamming',  # Moderate side lobe attenuation
    'Blackman': 'blackman'  # High side lobe attenuation but a wider transition band
}

# Prepare a dictionary to store frequency responses for each window type
frequency_responses = {}

# Generate filters and compute frequency responses for each window type
for window_name, window in window_types.items():
    h_D = firwin(filter_length, omega_c / np.pi, window=window)
    freqs, H_D = freqz(h_D, worN=512)
    magnitude_response = 20 * np.log10(np.abs(H_D))
    frequency_responses[window_name] = (freqs, magnitude_response)

# Plot frequency responses for each window
plt.figure(figsize=(12, 6))
for window_name, (freqs, magnitude_response) in frequency_responses.items():
    plt.plot(freqs / np.pi, magnitude_response, label=f"{window_name} Window")

plt.title("Magnitude Response of $h_D[n]$ with Different Window Functions (in dB)")
plt.xlabel("Normalized Frequency (×π rad/sample)")
plt.ylabel("Magnitude (dB)")
plt.legend()
plt.grid(True)
plt.ylim([-100, 10])  # Focus on the region showing attenuation differences
plt.show()

# Analysis: Determine stopband attenuation for each window type
for window_name, (freqs, magnitude_response) in frequency_responses.items():
    # Find minimum magnitude in the stopband (frequencies beyond cutoff frequency omega_c)
    stopband_magnitudes = magnitude_response[freqs >= omega_c]
    stop_att = np.min(stopband_magnitudes)
    print(f"Stopband attenuation for {window_name} Window: {stop_att:.2f} dB")


In [ ]:
#Siste oppgave på 2

In [ ]:
#last task of em all:

"# WRITE YOUR CODE IN THIS CELL:

from scipy.signal import convolve
from IPython.display import Audio
import numpy as np
import matplotlib.pyplot as plt

# Apply the complementary filter to the noisy signal x[n]
filtered_signal = convolve(xn, h_s, mode='same')

# Play the original and filtered audio signals for comparison
print("Original noisy signal:")
#Audio(xn, rate=fs)


# Calculate the envelopes by taking the absolute values and smoothing
envelope_original = np.abs(xn)
envelope_filtered = np.abs(filtered_signal)

# Plot envelopes of the original and filtered signals


plt.figure(figsize=(12, 6))

plt.subplot(2, 1, 1)
plt.plot(np.linspace(0, len(xn) / fs, len(xn), endpoint=False), xn)
plt.grid(True)
plt.title("Original Signal")
plt.xlabel("Time (s)")
plt.xlim([0, len(xn) / fs])

plt.subplot(2, 1, 2)
plt.plot(np.linspace(0, len(filtered_signal) / fs, len(filtered_signal), endpoint=False), xn)
plt.grid(True)
plt.title("Filtered Signal")
plt.xlabel("Time (s)")
plt.xlim([0, len(xn) / fs])

plt.tight_layout()
plt.show()

Audio(filtered_signal, rate=fs)
"